# Урок 1

Привет. В этом уроке мы научимся извлекать MFCC признаки из аудио файла с помощью python. Для этого необходимо сделать следующее:

1) Загрузить отсчеты и частоту дискретизации wav файла.<br>
2) Посчитать MFCC признаки по полученным выше данным.<br>
3) Записать посчитанные признаки в удобный формат.<br>

In [1]:
import numpy as np
import scipy.io.wavfile
from python_speech_features import mfcc

wav_name = 'example.wav'

# чтение отсчетов wav файла:
s_rate, samples = scipy.io.wavfile.read(wav_name)

# чтение MFCC признаков по отсчетам файла:
features = mfcc(samples, s_rate)

# запись признаков в бинарный файл example.npy:
features_file = wav_name.split('.')[0]
np.save(features_file, features)

В результате мы получили бинарный файл example.npy, содержащий в себе матрицу с 13-мерными MFCC признаками аудио файла example.wav

Но работать с бинарными файлами не всегда удобно. Мы будем пользоваться общепринятым ark текстовым форматом библиотеки распознавания речи KALDI, где в одном файле будут храниться признаки для всех используемых записей. Он выглядит следующим образом:

file1-identifier [<br>
vec0_0 vec0_1 vec0_2 ... vec0_12<br>
vec1_0 vec1_1 vec1_2 ... vec1_12<br>
....<br>
vecN_0 vecN_1 vecN_2 ... vecN_12<br>
]<br>
file2-identifier [<br>
vec0_0 vec0_1 vec0_2 ... vec0_12<br>
vec1_0 vec1_1 vec1_2 ... vec1_12<br>
....<br>
vecN_0 vecN_1 vecN_2 ... vecN_12<br>
]<br>

Здесь "file1-identifier" – это имя файла, а "vec0_0 vec0_1 vec0_2 ... vec0_12" – вектор MFCC признаков первого (нулевого) кадра записи. 
<br><b>Задание 1</b>.Сохранить нашу матрицу с признаками (features) в указанном формате: 

In [2]:
# запись признаков в формат ark,t:
file_name = features_file + '.txtftr'

#---------------------------TODO-----------------------------------------
with open(file_name, 'w') as fn:
    fn.write('{} [\n'.format(file_name))
    for ftr in features:
        fn.write(str(ftr).replace('\n', '')[1:-1] + '\n')
    fn.write(']')
#------------------------------------------------------------------------    

Для чтения признаков из такого формата мы будем использовать удобную читалку FtrFile. Функция FtrFile.FtrDirectoryReader принимает файл с признаками и возвращает имя файла и класс features, который имеет следующие атрибуты:

- nSamples – количество векторов в фале;<br>
- nDim – размерность вектора признаков;<br>
- readvec() – возвращает очередной вектор признаков;<br>
- getall() – возвращает все признаки текущего файла.<br>

Также, для ее работы необходимо добавлять префикс 'ark,t:' к названию файла с признаками.

Попробуем с ее помощью прочитать наш example.txtftr:

In [3]:
import FtrFile

ark_file = 'ark,t:' + file_name
for fileName, features in FtrFile.FtrDirectoryReader(ark_file):
    print('имя файла: {}'.format(fileName))
    print('количество векторов в файле: {}'.format(features.nSamples))
    print('размерность вектора признаков: {}'.format(features.nDim))
    print('вектор признаков первого кадра: {}'.format(features.readvec()))

имя файла: example.txtftr
количество векторов в файле: 361
размерность вектора признаков: 13
вектор признаков первого кадра: [  7.30046368 -15.96167278 -12.18360806  -1.89521074  -2.60937214
   4.60660887  -4.33287382   1.52098656 -18.3515892   -2.31159902
  -8.2139082    6.16862965  -1.04032767]


Для работы с признаками удобно делать цикл по количеству векторов в файле и в нем уже вызывать features.readvec() для получения признаков текущего кадра. Этим мы займемся в следующем уроке.